In [1]:
import time
i=time.time()

Fonte: Comexstat(http://comexstat.mdic.gov.br/pt/geral), portal para acesso gratuito às estatísticas de comércio exterior do Brasil, com os dados utilizados para construção da balança comercial brasileira, detalhados por NCM ou por Municípios do exportador/importador.

O site informa que os dados estão disponíveis em arquivos compactados que contém arquivos CSV, utilizando como separador o ponto e vírgula (;), e com os nomes das colunas na primeira linha. 

Para esse projeto, o interesse esta sobre a base de dados denominada IMP_COMPLETA.zip, que contém histórico mensal das importações detalhadas por produto (código NCM), Unidade Federativa importadora, ano, mês, país de origem, quilograma líquido, valor dólar FOB (US$) e outras.

Para aproveitamento completo dos arquivos de dados, será necessário baixar e utilizar as Tabelas Auxiliares contendo a "Correlações de Códigos e Classificações". 

O arquivo principal e as tabelas auxiliares foram baixadas e salvas localmente na pasta '~/git/BD/Comexstat/'.

A classificação de produto mais abrangente disponível é a SH2 (Sistema Harmonizado) e no nome dela se buscará identificar palavras como "fertilizantes" ou "adubo".

# Baixa zip

In [2]:
%run '/home/andre301267/git/BD/Comexstat/function/baixa_zip.ipynb'

**Função baixa_zip(url,local_filename) disponível**


In [3]:
url = 'https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/IMP_COMPLETA.zip'
local_filename = '/home/andre301267/git/BD/temp/IMP_COMPLETA.zip'

In [4]:
baixa_zip(url,local_filename)

Arquivo salvo como /home/andre301267/git/BD/temp/IMP_COMPLETA.zip


# Carrega tabelas complementares

In [5]:
%run '~/git/BD/Comexstat/function/Extrai-tab_complementares.ipynb'

Variáveis disponíveis: ncm, ncm_sh, pais, uf_mun, uf, via, urf, ncm_unidade


Reconhecer os códigos mais abrangentes dos produtos de interesse no sistema internacional SH:

# Identifica código foco

In [6]:
co_sh2_fert=ncm_sh[ncm_sh.NO_SH2_POR.str.lower().str.contains('fertilizante')]['CO_SH2'].unique()
co_sh2_fert

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


array([31])

Lista os códigos SH6 contidos no código SH2 de fertilizantes:

In [7]:
ncm_sh_list=ncm_sh[ncm_sh.CO_SH2.isin(co_sh2_fert)]['CO_SH6'].unique()
ncm_sh_list

array([310100, 310210, 310221, 310229, 310230, 310240, 310250, 310260,
       310270, 310280, 310290, 310310, 310311, 310319, 310320, 310390,
       310410, 310420, 310430, 310490, 310510, 310520, 310530, 310540,
       310551, 310559, 310560, 310590])

Lê tabela com código comum ao Mercosul, que é o de maior especificação de produto.

Na última tabela de NCM, localizamos os códigos NCM que se relacionam aos SH6 de fertilizantes.

In [8]:
list_ncm_fert=ncm[ncm.CO_SH6.isin(ncm_sh_list)].CO_NCM.unique()
list_ncm_fert

array([31053000, 31031100, 31031900, 31059011, 31059019, 31059090,
       31031020, 31031030, 31032000, 31039011, 31053010, 31053090,
       31054000, 31055100, 31055900, 31056000, 31039019, 31039090,
       31041000, 31042010, 31042090, 31043010, 31043090, 31049010,
       31049090, 31051000, 31052000, 31010000, 31021010, 31021090,
       31022100, 31022910, 31022990, 31023000, 31024000, 31025011,
       31025019, 31025090, 31026000, 31027000, 31028000, 31029000,
       31031010])

Identificando os nomes dos arquivos no interior do IMP_COMPLETA.zip

In [9]:
%run '/home/andre301267/git/BD/Comexstat/function/Id_internal_zipfile.ipynb'

** Função id_internal_zipfile(local_filename)disponível **


In [10]:
internal_zipfile=id_internal_zipfile(local_filename)

IMP_COMPLETA.csv


# Visualiza primeiras linhas do interno

In [11]:
%run '/home/andre301267/git/BD/Comexstat/function/vslz_primeiras_linhas.ipynb'

** Função vslz_primeiras_linhas(local_filename,internal_zipfile) disponível**


In [12]:
vslz_primeiras_linhas(local_filename,internal_zipfile)

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO
0,1997,9,29251990,10,87,SP,1,817800,200,200,4888,180,24
1,1997,8,85049010,10,399,SP,4,817600,4891,84,30116,409,93
2,1997,8,54023300,10,63,SP,7,1010900,143834,143834,402427,10121,266
3,1997,3,38121000,10,249,SP,1,817800,9434,9172,61190,3011,163
4,1997,10,73209000,10,399,SP,4,817600,222510,49,3638,456,18


# Extrai filtrando

In [13]:
%run '/home/andre301267/git/BD/Comexstat/function/extrai_filtrando_ncm.ipynb'

** Função extrai_filtrando_ncm(local_filename,internal_zipfile) disponível **


In [14]:
imp_fert=extrai_filtrando_ncm(local_filename,internal_zipfile)

df imp_fert disponível


# Corrige Columns Name

In [15]:
imp_fert=imp_fert.rename(columns={'SG_UF_NCM':'UF'})

# Salva arquivo extraído e filtrado

In [16]:
imp_fert.to_csv('~/git/BD/temp/imp_fert_ncm.csv', index=False)

In [17]:
import os

# Verificar se o arquivo existe antes de tentar excluí-lo
if os.path.exists(local_filename):
    os.remove(local_filename)
    print(f'O arquivo {local_filename} foi excluído com sucesso.')
else:
    print(f'O arquivo {local_filename} não foi encontrado.')

O arquivo /home/andre301267/git/BD/temp/IMP_COMPLETA.zip foi excluído com sucesso.


In [18]:
imp_fert

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,UF,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO
266,1997,12,31042090,10,23,RS,1,1015400,7130535,7130535,922261,85567,1935
874,1997,3,31027000,10,249,SP,1,817800,12,6,103,4,0
1095,1997,11,31042090,10,249,SP,4,817600,12,1,1197,35,6
1543,1997,12,31021010,10,831,SC,1,920151,4507463,4507463,549976,54024,0
2273,1997,3,31022100,10,23,RJ,1,717600,144,7200,15582,681,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41638389,2024,1,31031100,10,474,PR,1,917800,5637000,5637000,2155422,106875,3753
41639100,2024,6,31023000,10,676,RS,1,1017700,5128932,5128932,1159272,174250,1134
41639307,2024,3,31042090,10,676,GO,1,317903,2000000,2000000,472000,88000,0
41639425,2024,2,31021010,10,160,ES,1,717600,588000,588000,233340,54600,251


In [19]:
f=time.time()
print(f"Tempo de execução: {round((f-i)/60,1)} minutos.")

Tempo de execução: 3.3 minutos.
